In [1]:
import pandas as pd

df = pd.read_pickle('suumo_bunkyo.pkl')

In [2]:
df.head()

,マンション名,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積
0,Ｋｍマンション,東京都文京区本駒込５,ＪＲ山手線/駒込駅 歩6分,ＪＲ山手線/田端駅 歩15分,都営三田線/千石駅 歩17分,築26年,5階建,3階,12万円,5000円,"12万円,12万円",2LDK,44.95m2
1,ダイアパレス文京サファイアレジデンス,東京都文京区水道２,東京メトロ有楽町線/江戸川橋駅 歩6分,東京メトロ東西線/神楽坂駅 歩10分,ＪＲ総武線/飯田橋駅 歩12分,築19年,9階建,9階,27.5万円,15000円,"55万円,27.5万円",3LDK,73.26m2
2,ダイアパレス文京サファイアレジデンス,東京都文京区水道２,東京メトロ有楽町線/江戸川橋駅 歩6分,東京メトロ東西線/神楽坂駅 歩10分,ＪＲ総武線/飯田橋駅 歩12分,築19年,9階建,9階,27.5万円,15000円,"55万円,27.5万円",3LDK,73.26m2
3,本駒込シティハイツ,東京都文京区本駒込１,東京メトロ南北線/本駒込駅 歩6分,都営三田線/白山駅 歩7分,都営三田線/千石駅 歩9分,築30年,3階建,2階,12.2万円,4000円,"24.4万円,-",3DK,54.81m2
4,本駒込シティハイツ,東京都文京区本駒込１,東京メトロ南北線/本駒込駅 歩6分,都営三田線/白山駅 歩7分,都営三田線/千石駅 歩9分,築30年,3階建,2階,12.2万円,4000円,"24.4万円,-",3DK,54.81m2


In [3]:
# 住所から文京区内の地区情報のみを取り出す
df['地区'] = df['住所'].str.replace('(東京都文京区|[１-９])', '')
df = df.drop('住所', axis=1)

In [4]:
# 地区を one-hot encoding
for area in df['地区'].unique():
    df['地区:' + area] = (df['地区'] == area).astype(int)
df = df.drop('地区', axis=1)

In [5]:
# 築年数を数字に変換する
df['築年数'] = df['築年数'].str.replace('新築', '0').str.replace('(築|年|新)', '').astype('int')

In [6]:
# 建物高さを数字に変換する
df['建物高さ'] = df['建物高さ'].str.replace('階建', '').str.replace('地下\d地上', '').astype('int')

In [7]:
# 階を数字に変換する
# 2-3 のような場合は高い階に合わせる
# 地下はマイナス扱い
# 不明な階は1階扱い
df['階'] = df['階'].str.replace('^.*-', '').str.replace('B', '-').str.replace('階', '').str.replace('^$', '1').astype('int')

In [8]:
# 専有面積を数字に変換する
df['専有面積'] = df['専有面積'].str.replace('m2', '').astype('float')

In [9]:
# 賃料を数字に変換する
df['賃料'] = (df['賃料'].str.replace('万円', '').astype('float') * 10000).astype('int')
df['管理費'] = df['管理費'].str.replace('-', '0').str.replace('円', '').astype('int')
df['賃料'] = df['賃料'] + df['管理費']
df = df.drop('管理費', axis=1)

In [10]:
splitted = df['敷/礼/保証/敷引,償却'].str.split(',', expand=True)
splitted[0] = (splitted[0].str.replace('-', '0万円').str.replace('万円', '').astype(float) * 10000).astype(int)
splitted[1] = (splitted[1].str.replace('-', '0万円').str.replace('万円', '').astype(float) * 10000).astype(int)
df['初期費用'] = splitted[0] + splitted[1]
df = df.drop('敷/礼/保証/敷引,償却', axis=1)

In [11]:
# 間取りを変換する
d = df['間取り'].str.replace('ワンルーム', '1K')

df['部屋数'] = d.str.replace('[A-Z]', '').astype('int')
df['間取りDK'] = d.str.contains('DK').astype('int')
df['間取りK'] = (df['間取りDK'].astype('bool') ^ d.str.contains('K')).astype(int)
df['間取りL'] = d.str.contains('L').astype('int')
df['間取りS'] = d.str.contains('S').astype('int')
df = df.drop('間取り', axis=1)

In [31]:
# 駅を count encoding 重要な駅ほど参考に書かれやすいため
for i in range(1, 4):
    d = df[f'立地{i}'].str.replace('\s.*$', '').str.replace('^.*/', '')
    cnt = d.value_counts().to_dict()
    cnt[''] = 0
    df[f'駅{i}'] = d.map(lambda x: cnt[x])
    d = df[f'立地{i}'].str.replace('^.*歩', '').str.replace('分$', '')
    d.replace('None', value=0, inplace=True)
    df[f'徒歩{i}'] = d.astype(int)
    df = df.drop(f'立地{i}', axis=1)

In [32]:
df.head()

,マンション名,築年数,建物高さ,階,賃料,専有面積,地区:本駒込,地区:水道,地区:春日,地区:千駄木,...,間取りDK,間取りK,間取りL,間取りS,駅1,徒歩1,駅2,徒歩2,駅3,徒歩3
0,Ｋｍマンション,26,5,3,125000,44.95,1,0,0,0,...,1,0,1,0,368,6,99,15,404,17
1,ダイアパレス文京サファイアレジデンス,19,9,9,290000,73.26,0,1,0,0,...,1,0,1,0,661,6,412,10,191,12
2,ダイアパレス文京サファイアレジデンス,19,9,9,290000,73.26,0,1,0,0,...,1,0,1,0,661,6,412,10,191,12
3,本駒込シティハイツ,30,3,2,126000,54.81,1,0,0,0,...,1,0,0,0,365,6,622,7,404,9
4,本駒込シティハイツ,30,3,2,126000,54.81,1,0,0,0,...,1,0,0,0,365,6,622,7,404,9


In [50]:
test = df.sample(500)
train = df.drop(test.index.values).reset_index()
test = test.reset_index()
print(len(train), len(test))

8773 500


In [51]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')